### Installation

In [2]:
%pip install transformers==4.44.2 
%pip install langchain==0.2.16 
%pip install langchain-community==0.2.16 
%pip install langchain-core==0.2.38 
%pip install langchain-huggingface==0.0.3
%pip install langchain-text-splitters==0.2.4 
%pip install pypdf==5.0.1 
%pip install faiss-cpu==1.9.0




  Using cached langchain_community-0.2.16-py3-none-any.whl.metadata (2.7 kB)
  Using cached dataclasses_json-0.6.7-py3-none-any.whl.metadata (25 kB)
  Using cached marshmallow-3.23.0-py3-none-any.whl.metadata (7.6 kB)
  Using cached typing_inspect-0.9.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached mypy_extensions-1.0.0-py3-none-any.whl.metadata (1.1 kB)
Using cached langchain_community-0.2.16-py3-none-any.whl (2.3 MB)
Using cached dataclasses_json-0.6.7-py3-none-any.whl (28 kB)
Using cached marshmallow-3.23.0-py3-none-any.whl (49 kB)
Using cached typing_inspect-0.9.0-py3-none-any.whl (8.8 kB)
Using cached mypy_extensions-1.0.0-py3-none-any.whl (4.7 kB)
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.2.41
    Uninstalling langchain-core-0.2.41:
      Successfully uninstalled langchain-core-0.2.41
Note: you may need to restart the kernel to use updated packages.
^C
Note: you may need to restart the kernel to use updated packages.

Note: yo

  Using cached jinja2-3.1.4-py3-none-any.whl.metadata (2.6 kB)
   ---------------------------------------- 0.0/203.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/203.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/203.1 MB ? eta -:--:--
   ---------------------------------------- 0.3/203.1 MB ? eta -:--:--
   ---------------------------------------- 0.3/203.1 MB ? eta -:--:--
   ---------------------------------------- 0.3/203.1 MB ? eta -:--:--
   ---------------------------------------- 0.5/203.1 MB 316.6 kB/s eta 0:10:40
   ---------------------------------------- 0.5/203.1 MB 316.6 kB/s eta 0:10:40
   ---------------------------------------- 0.5/203.1 MB 316.6 kB/s eta 0:10:40
   ---------------------------------------- 0.8/203.1 MB 342.4 kB/s eta 0:09:51
   ---------------------------------------- 0.8/203.1 MB 342.4 kB/s eta 0:09:51
   ---------------------------------------- 1.0/203.1 MB 393.2 kB/s eta 0:08:34
   ----------------------------

In [3]:
!pip install pymongo

In [5]:
from pymongo import MongoClient # Connessione al server MongoDB 
client = MongoClient('localhost', 27017) # Selezione del database e della collection 
db = client['test'] 
collection = db['test']

### Funzioni Mongo - Python

In [6]:
def inserisci_documento(documento):
    result = collection.insert_one(documento)
    return result.inserted_id
 
# Esempio di utilizzo
documento = {
    "nome": "Mario Rossi",
    "età": 30,
    "città": "Roma"
}
id_documento = inserisci_documento(documento)
print(f"Documento inserito con ID: {id_documento}")

Documento inserito con ID: 6717b35127b3a45fcbdff3de


In [7]:
def leggi_documento(filtro):
    documento = collection.find_one(filtro)
    return documento
 
# Esempio di utilizzo
filtro = {"nome": "Mario Rossi"}
documento = leggi_documento(filtro)
print(f"Documento trovato: {documento}")

Documento trovato: {'_id': ObjectId('6717b35127b3a45fcbdff3de'), 'nome': 'Mario Rossi', 'età': 30, 'città': 'Roma'}


In [8]:
def aggiorna_documento(filtro, nuovi_dati):
    result = collection.update_one(filtro, {"$set": nuovi_dati})
    return result.modified_count
 
# Esempio di utilizzo
filtro = {"nome": "Mario Rossi"}
nuovi_dati = {"età": 31}
conteggio_modifiche = aggiorna_documento(filtro, nuovi_dati)
print(f"Numero di documenti aggiornati: {conteggio_modifiche}")

Numero di documenti aggiornati: 1


### Lettura pdf

In [16]:
from langchain_community.document_loaders import PyPDFLoader

In [18]:
loader = PyPDFLoader("./files_pdf/thinkpython2.pdf")
loaded_data = loader.load_and_split()
loaded_data

[Document(metadata={'source': './files_pdf/thinkpython2.pdf', 'page': 0}, page_content='Think Python\nHow to Think Like a Computer Scientist\n2nd Edition, Version 2.4.0'),
 Document(metadata={'source': './files_pdf/thinkpython2.pdf', 'page': 2}, page_content='Think Python\nHow to Think Like a Computer Scientist\n2nd Edition, Version 2.4.0\nAllen Downey\nGreen Tea Press\nNeedham, Massachusetts'),
 Document(metadata={'source': './files_pdf/thinkpython2.pdf', 'page': 3}, page_content='Copyright © 2015 Allen Downey.\nGreen Tea Press\n9 Washburn Ave\nNeedham MA 02492\nPermission is granted to copy, distribute, and/or modify this document under the terms of the\nCreative Commons Attribution-NonCommercial 3.0 Unported License, which is available at http:\n//creativecommons.org/licenses/by-nc/3.0/ .\nThe original form of this book is L ATEX source code. Compiling this L ATEX source has the effect of gen-\nerating a device-independent representation of a textbook, which can be converted to othe

In [19]:
[i.replace(' ','') for i in loaded_data[0].page_content.split('\n')]

['ThinkPython', 'HowtoThinkLikeaComputerScientist', '2ndEdition,Version2.4.0']

Split in chunck significativi

In [25]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
text = loaded_data[74].page_content
 
 
# Dizionario di sostituzioni
replacements = {
    '\n\n\n': '\n\n',  # Normalizza i ritorni a capo multipli (ad esempio, da 3 a 2)
    '\n\n': '\n\n',    # Mantieni i doppi ritorni per separare i paragrafi
    '-\n': '',         # Unisci parole spezzate alla fine della riga (es. "Python-\nista" diventa "Pythonista")
    '\n ': '\n',       # Rimuovi spazi extra dopo i ritorni a capo
    ' \n': '',         # Rimuovi ritorni a capo tra parole che non sono separate correttamente
    '\n': ' ',         # Converti singoli ritorni a capo in spazi per mantenere la continuità del testo
}
 
# Replace each character in the dictionary
for old, new in replacements.items():
    text = text.replace(old, new)
 
 
# text splitter
text_splitter = RecursiveCharacterTextSplitter(
 
    chunk_size=545,
    chunk_overlap=50,
    length_function=len,
    # separators=[' '],
    is_separator_regex=False,
)
 
texts = text_splitter.create_documents([text])
texts

[Document(page_content='54 Chapter 6. Fruitful functions 1. Start with a working program and make small incremental changes. At any point, if there is an error, you should have a good idea where it is. 2. Use variables to hold intermediate values so you can display and check them. 3. Once the program is working, you might want to remove some of the scaffolding or consolidate multiple statements into compound expressions, but only if it does not make the program difﬁcult to read. As an exercise, use incremental development to write a function called hypotenuse'),
 Document(page_content='development to write a function called hypotenuse that returns the length of the hypotenuse of a right triangle given the lengths of the other two legs as arguments. Record each stage of the development process as you go. 6.3 Composition As you should expect by now, you can call one function from within another. As an example, we’ll write a function that takes two points, the center of the circle and a p

In [26]:
for i in texts:
    print(len(i.page_content))

541
539
538
493


Salvataggio dei chunk su MongoDB

In [ ]:
#embedding
from langchain_community.embeddings import OllamaEmbeddings

